# 3 Bus (2 Generators, 1 Load) Steady state SP vs DP

In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import dpsimpy

#%matplotlib widget

## DP Simulation

### Parameters

In [ ]:
#-----------Power system-----------
#Voltage level as Base Voltage
V_nom = 230e3

#-----------Generator 1 (bus1)-----------
#Machine parameters
nom_power_G1 = 300e6
nom_ph_ph_volt_RMS_G1=25e3
nom_freq_G1 = 60
H_G1 = 6
Xpd_G1 = 0.3 # in p.u
Rs_G1 = 0
Kd_G1 = 1.5
# Initialization parameters 
init_active_power_G1 = 270e6
init_mech_power_G1 = 270e6
set_point_voltage_G1 = nom_ph_ph_volt_RMS_G1 * 1.05

#-----------Generator 2 (bus2)-----------
#Machine parameters
nom_power_G2 = 50e6
nom_ph_ph_volt_RMS_G2=13.8e3
nom_freq_G2 = 60
H_G2 = 2
Xpd_G2 = 0.1 # in p.u
Rs_G2 = 0
Kd_G2 = 1.5
# Initialization parameters 
init_active_power_G2 = 45e6
init_mech_power_G2 = 45e6
set_point_voltage_G2 = nom_ph_ph_volt_RMS_G2 * 0.95

#-----------Transformers-----------
t1_ratio = V_nom / nom_ph_ph_volt_RMS_G1
t2_ratio = V_nom / nom_ph_ph_volt_RMS_G2

#-----------Load (bus3)-----------
active_power_L = 310e6
reactive_power_L = 150e6

#-----------Transmission Lines-----------
#PiLine parameters
#line 1-2 (180km)
line_resistance12 = 0.04*180
line_inductance12 = (0.4/377)*180
line_capacitance12 = (4.3e-6/377)*180
line_conductance12 = 0
#line 1-3 (150km)
line_resistance13 = 0.0267*150
line_inductance13 = (0.267/377)*150
line_capacitance13 = (4.3e-6/377)*150
line_conductance13 = 0
#line 2-3 (80km)
line_resistance23 = 0.04*80
line_inductance23 = (0.267/377)*80
line_capacitance23 = (4.3e-6/377)*80
line_conductance23 = 0

#Switch to trigger fault at generator terminal
switch_open = 1e12
switch_closed = 0.1

#Simulation parameters
sim_name='DP_SynGenTrStab_3Bus_SteadyState'
final_time = 10
time_step = 0.001
scale_inertia= 1.0

### Powerflow for Initialization

In [ ]:
time_step_pf = final_time
final_time_pf = final_time + time_step_pf
sim_name_pf = sim_name + "_PF"
dpsimpy.Logger.set_log_dir("logs/" + sim_name_pf)

#Components
n1_pf = dpsimpy.sp.SimNode("n1", dpsimpy.PhaseType.Single)
n2_pf = dpsimpy.sp.SimNode("n2", dpsimpy.PhaseType.Single)
n3_pf = dpsimpy.sp.SimNode("n3", dpsimpy.PhaseType.Single)

#Synchronous generator 1
gen1_pf = dpsimpy.sp.ph1.SynchronGenerator("SynGen1", dpsimpy.LogLevel.debug)
#setPointVoltage is defined as the voltage at the transfomer primary side and should be transformed to network side
gen1_pf.set_parameters(nom_power_G1, nom_ph_ph_volt_RMS_G1, init_active_power_G1,
                       set_point_voltage_G1 * t1_ratio, dpsimpy.PowerflowBusType.VD)
gen1_pf.set_base_voltage(V_nom)

#Synchronous generator 2
gen2_pf = dpsimpy.sp.ph1.SynchronGenerator("SynGen2", dpsimpy.LogLevel.debug)
#setPointVoltage is defined as the voltage at the transfomer primary side and should be transformed to network side
gen2_pf.set_parameters(nom_power_G2, nom_ph_ph_volt_RMS_G2, init_active_power_G2,
                       set_point_voltage_G2 * t2_ratio, dpsimpy.PowerflowBusType.PV)
gen2_pf.set_base_voltage(V_nom)

#use Shunt as Load for powerflow
load_pf = dpsimpy.sp.ph1.Shunt("Load", dpsimpy.LogLevel.debug)
load_pf.set_parameters(active_power_L / V_nom**2, - reactive_power_L / V_nom**2)
load_pf.set_base_voltage(V_nom)

#Line12
line12_pf = dpsimpy.sp.ph1.PiLine("PiLine12", dpsimpy.LogLevel.debug)
line12_pf.set_parameters(line_resistance12, line_inductance12, line_capacitance12, line_conductance12)
line12_pf.set_base_voltage(V_nom)

#Line13
line13_pf = dpsimpy.sp.ph1.PiLine("PiLine13", dpsimpy.LogLevel.debug)
line13_pf.set_parameters(line_resistance13, line_inductance13, line_capacitance13, line_conductance13)
line13_pf.set_base_voltage(V_nom)

#Line23
line23_pf = dpsimpy.sp.ph1.PiLine("PiLine23", dpsimpy.LogLevel.debug)
line23_pf.set_parameters(line_resistance23, line_inductance23, line_capacitance23, line_conductance23)
line23_pf.set_base_voltage(V_nom)

# Topology
gen1_pf.connect([n1_pf])
gen2_pf.connect([n2_pf])
load_pf.connect([n3_pf])
line12_pf.connect([n1_pf, n2_pf])
line13_pf.connect([n1_pf, n3_pf])
line23_pf.connect([n2_pf, n3_pf])
system_pf = dpsimpy.SystemTopology(60, [n1_pf, n2_pf, n3_pf], [gen1_pf, gen2_pf, load_pf, line12_pf, line13_pf, line23_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute("v_bus1", "v", n1_pf)
logger_pf.log_attribute("v_bus2", "v", n2_pf)
logger_pf.log_attribute("v_bus3", "v", n3_pf)

# Simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(time_step_pf)
sim_pf.set_final_time(final_time_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.add_logger(logger_pf)
sim_pf.run()

### Dynamic Simulation

In [ ]:
sim_name_dp = sim_name + "_DP"
dpsimpy.Logger.set_log_dir("logs/" + sim_name_dp)

#Components
n1_dp = dpsimpy.dp.SimNode("n1", dpsimpy.PhaseType.Single)
n2_dp = dpsimpy.dp.SimNode("n2", dpsimpy.PhaseType.Single)
n3_dp = dpsimpy.dp.SimNode("n3", dpsimpy.PhaseType.Single)

#Synchronous generator 1
gen1_dp = dpsimpy.dp.ph1.SynchronGeneratorTrStab("SynGen1", dpsimpy.LogLevel.debug)
# Xpd is given in p.u of generator base at transfomer primary side and should be transformed to network side
gen1_dp.set_standard_parameters_PU(nom_power_G1, nom_ph_ph_volt_RMS_G1, nom_freq_G1,
                                  Xpd_G1 * t1_ratio**2, scale_inertia * H_G1, Rs_G1,
                                  Kd_G1)

#Synchronous generator 2
gen2_dp = dpsimpy.dp.ph1.SynchronGeneratorTrStab("SynGen2", dpsimpy.LogLevel.debug)
# Xpd is given in p.u of generator base at transfomer primary side and should be transformed to network side
gen2_dp.set_standard_parameters_PU(nom_power_G2, nom_ph_ph_volt_RMS_G2, nom_freq_G2,
                                  Xpd_G2 * t2_ratio**2, scale_inertia * H_G2, Rs_G2,
                                  Kd_G2)

# Load
load_dp = dpsimpy.dp.ph1.RXLoad("Load", dpsimpy.LogLevel.debug)
load_dp.set_parameters(active_power_L, reactive_power_L, V_nom)

#Line12
line12_dp = dpsimpy.dp.ph1.PiLine("PiLine12", dpsimpy.LogLevel.debug)
line12_dp.set_parameters(line_resistance12, line_inductance12, line_capacitance12, line_conductance12)

#Line13
line13_dp = dpsimpy.dp.ph1.PiLine("PiLine13", dpsimpy.LogLevel.debug)
line13_dp.set_parameters(line_resistance13, line_inductance13, line_capacitance13, line_conductance13)

#Line23
line23_dp = dpsimpy.dp.ph1.PiLine("PiLine23", dpsimpy.LogLevel.debug)
line23_dp.set_parameters(line_resistance23, line_inductance23, line_capacitance23, line_conductance23)

# Topology
gen1_dp.connect([n1_dp])
gen2_dp.connect([n2_dp])
load_dp.connect([n3_dp])
line12_dp.connect([n1_dp, n2_dp])
line13_dp.connect([n1_dp, n3_dp])
line23_dp.connect([n2_dp, n3_dp])
system_dp = dpsimpy.SystemTopology(60, [n1_dp, n2_dp, n3_dp], [gen1_dp, gen2_dp, load_dp, line12_dp, line13_dp, line23_dp])

# Initialization of dynamic topology
system_dp.init_with_powerflow(system_pf)

# Logging
logger_dp = dpsimpy.Logger(sim_name_dp)
logger_dp.log_attribute("v1", "v", n1_dp)
logger_dp.log_attribute("v2", "v", n2_dp)
logger_dp.log_attribute("v3", "v", n3_dp)

logger_dp.log_attribute("v_line12", "v_intf", line12_dp)
logger_dp.log_attribute("v_line13", "v_intf", line13_dp)
logger_dp.log_attribute("v_line23", "v_intf", line23_dp)
logger_dp.log_attribute("i_line12", "i_intf", line12_dp)
logger_dp.log_attribute("i_line13", "i_intf", line13_dp)
logger_dp.log_attribute("i_line23", "i_intf", line23_dp)

logger_dp.log_attribute("Ep_gen1", "Ep_mag", gen1_dp)
logger_dp.log_attribute("v_gen1", "v_intf", gen1_dp)
logger_dp.log_attribute("i_gen1", "i_intf", gen1_dp)
logger_dp.log_attribute("wr_gen1", "w_r", gen1_dp)
logger_dp.log_attribute("delta_gen1", "delta_r", gen1_dp)

logger_dp.log_attribute("Ep_gen2", "Ep_mag", gen2_dp)
logger_dp.log_attribute("v_gen2", "v_intf", gen2_dp)
logger_dp.log_attribute("i_gen2", "i_intf", gen2_dp)
logger_dp.log_attribute("wr_gen2", "w_r", gen2_dp)
logger_dp.log_attribute("delta_gen2", "delta_r", gen2_dp)

logger_dp.log_attribute("P_elec2", "P_elec", gen2_dp)


# Simulation
sim_dp = dpsimpy.Simulation(sim_name_dp, dpsimpy.LogLevel.debug)
sim_dp.set_system(system_dp)
sim_dp.set_time_step(time_step)
sim_dp.set_final_time(final_time)
sim_dp.set_domain(dpsimpy.Domain.DP)
sim_dp.add_logger(logger_dp)

sim_dp.run()

## SP Simulation

### Parameters

In [ ]:
#-----------Power system-----------
#Voltage level as Base Voltage
V_nom = 230e3

#-----------Generator 1 (bus1)-----------
#Machine parameters
nom_power_G1 = 300e6
nom_ph_ph_volt_RMS_G1=25e3
nom_freq_G1 = 60
H_G1 = 6
Xpd_G1 = 0.3 # in p.u
Rs_G1 = 0
Kd_G1 = 1.5
# Initialization parameters 
init_active_power_G1 = 270e6
init_mech_power_G1 = 270e6
set_point_voltage_G1 = nom_ph_ph_volt_RMS_G1 * 1.05

#-----------Generator 2 (bus2)-----------
#Machine parameters
nom_power_G2 = 50e6
nom_ph_ph_volt_RMS_G2=13.8e3
nom_freq_G2 = 60
H_G2 = 2
Xpd_G2 = 0.1 # in p.u
Rs_G2 = 0
Kd_G2 = 1.5
# Initialization parameters 
init_active_power_G2 = 45e6
init_mech_power_G2 = 45e6
set_point_voltage_G2 = nom_ph_ph_volt_RMS_G2 * 0.95

#-----------Transformers-----------
t1_ratio = V_nom / nom_ph_ph_volt_RMS_G1
t2_ratio = V_nom / nom_ph_ph_volt_RMS_G2

#-----------Load (bus3)-----------
active_power_L = 310e6
reactive_power_L = 150e6

#-----------Transmission Lines-----------
#PiLine parameters
#line 1-2 (180km)
line_resistance12 = 0.04*180
line_inductance12 = (0.4/377)*180
line_capacitance12 = (4.3e-6/377)*180
line_conductance12 = 0
#line 1-3 (150km)
line_resistance13 = 0.0267*150
line_inductance13 = (0.267/377)*150
line_capacitance13 = (4.3e-6/377)*150
line_conductance13 = 0
#line 2-3 (80km)
line_resistance23 = 0.04*80
line_inductance23 = (0.267/377)*80
line_capacitance23 = (4.3e-6/377)*80
line_conductance23 = 0

#Switch to trigger fault at generator terminal
switch_open = 1e12
switch_closed = 0.1

#Simulation parameters
sim_name="SP_SynGenTrStab_3Bus_SteadyState"
final_time = 10
time_step = 0.001
scale_inertia= 1.0

### Powerflow for Initialization

In [ ]:
time_step_pf = final_time
final_time_pf = final_time + time_step_pf
sim_name_pf = sim_name + "_PF"
dpsimpy.Logger.set_log_dir("logs/" + sim_name_pf)

#Components
n1_pf = dpsimpy.sp.SimNode("n1", dpsimpy.PhaseType.Single)
n2_pf = dpsimpy.sp.SimNode("n2", dpsimpy.PhaseType.Single)
n3_pf = dpsimpy.sp.SimNode("n3", dpsimpy.PhaseType.Single)

#Synchronous generator 1
gen1_pf = dpsimpy.sp.ph1.SynchronGenerator("SynGen1", dpsimpy.LogLevel.debug)
#setPointVoltage is defined as the voltage at the transfomer primary side and should be transformed to network side
gen1_pf.set_parameters(nom_power_G1, nom_ph_ph_volt_RMS_G1, init_active_power_G1,
                       set_point_voltage_G1 * t1_ratio, dpsimpy.PowerflowBusType.VD)
gen1_pf.set_base_voltage(V_nom)

#Synchronous generator 2
gen2_pf = dpsimpy.sp.ph1.SynchronGenerator("SynGen2", dpsimpy.LogLevel.debug)
#setPointVoltage is defined as the voltage at the transfomer primary side and should be transformed to network side
gen2_pf.set_parameters(nom_power_G2, nom_ph_ph_volt_RMS_G2, init_active_power_G2,
                       set_point_voltage_G2 * t2_ratio, dpsimpy.PowerflowBusType.PV)
gen2_pf.set_base_voltage(V_nom)

#use Shunt as Load for powerflow
load_pf = dpsimpy.sp.ph1.Shunt("Load", dpsimpy.LogLevel.debug)
load_pf.set_parameters(active_power_L / V_nom**2, - reactive_power_L / V_nom**2)
load_pf.set_base_voltage(V_nom)

#Line12
line12_pf = dpsimpy.sp.ph1.PiLine("PiLine12", dpsimpy.LogLevel.debug)
line12_pf.set_parameters(line_resistance12, line_inductance12, line_capacitance12, line_conductance12)
line12_pf.set_base_voltage(V_nom)

#Line13
line13_pf = dpsimpy.sp.ph1.PiLine("PiLine13", dpsimpy.LogLevel.debug)
line13_pf.set_parameters(line_resistance13, line_inductance13, line_capacitance13, line_conductance13)
line13_pf.set_base_voltage(V_nom)

#Line23
line23_pf = dpsimpy.sp.ph1.PiLine("PiLine23", dpsimpy.LogLevel.debug)
line23_pf.set_parameters(line_resistance23, line_inductance23, line_capacitance23, line_conductance23)
line23_pf.set_base_voltage(V_nom)

# Topology
gen1_pf.connect([n1_pf])
gen2_pf.connect([n2_pf])
load_pf.connect([n3_pf])
line12_pf.connect([n1_pf, n2_pf])
line13_pf.connect([n1_pf, n3_pf])
line23_pf.connect([n2_pf, n3_pf])
system_pf = dpsimpy.SystemTopology(60, [n1_pf, n2_pf, n3_pf], [gen1_pf, gen2_pf, load_pf, line12_pf, line13_pf, line23_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute("v_bus1", "v", n1_pf)
logger_pf.log_attribute("v_bus2", "v", n2_pf)
logger_pf.log_attribute("v_bus3", "v", n3_pf)

# Simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(time_step_pf)
sim_pf.set_final_time(final_time_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.add_logger(logger_pf)
sim_pf.run()

### Dynamic Simulation

In [ ]:
sim_name_sp = sim_name + "_SP"
dpsimpy.Logger.set_log_dir("logs/" + sim_name_sp)

#Components
n1_sp = dpsimpy.sp.SimNode("n1", dpsimpy.PhaseType.Single)
n2_sp = dpsimpy.sp.SimNode("n2", dpsimpy.PhaseType.Single)
n3_sp = dpsimpy.sp.SimNode("n3", dpsimpy.PhaseType.Single)

#Synchronous generator 1
gen1_sp = dpsimpy.sp.ph1.SynchronGeneratorTrStab("SynGen1", dpsimpy.LogLevel.debug)
# Xpd is given in p.u of generator base at transfomer primary side and should be transformed to network side
gen1_sp.set_standard_parameters_PU(nom_power_G1, nom_ph_ph_volt_RMS_G1, nom_freq_G1,
                                  Xpd_G1 * t1_ratio**2, scale_inertia * H_G1, Rs_G1,
                                  Kd_G1)

#Synchronous generator 2
gen2_sp = dpsimpy.sp.ph1.SynchronGeneratorTrStab("SynGen2", dpsimpy.LogLevel.debug)
# Xpd is given in p.u of generator base at transfomer primary side and should be transformed to network side
gen2_sp.set_standard_parameters_PU(nom_power_G2, nom_ph_ph_volt_RMS_G2, nom_freq_G2,
                                  Xpd_G2 * t2_ratio**2, scale_inertia * H_G2, Rs_G2,
                                  Kd_G2)

# Load
load_sp = dpsimpy.sp.ph1.Load("Load", dpsimpy.LogLevel.debug)
load_sp.set_parameters(active_power_L, reactive_power_L, V_nom)

#Line12
line12_sp = dpsimpy.sp.ph1.PiLine("PiLine12", dpsimpy.LogLevel.debug)
line12_sp.set_parameters(line_resistance12, line_inductance12, line_capacitance12, line_conductance12)

#Line13
line13_sp = dpsimpy.sp.ph1.PiLine("PiLine13", dpsimpy.LogLevel.debug)
line13_sp.set_parameters(line_resistance13, line_inductance13, line_capacitance13, line_conductance13)

#Line23
line23_sp = dpsimpy.sp.ph1.PiLine("PiLine23", dpsimpy.LogLevel.debug)
line23_sp.set_parameters(line_resistance23, line_inductance23, line_capacitance23, line_conductance23)

# Topology
gen1_sp.connect([n1_sp])
gen2_sp.connect([n2_sp])
load_sp.connect([n3_sp])
line12_sp.connect([n1_sp, n2_sp])
line13_sp.connect([n1_sp, n3_sp])
line23_sp.connect([n2_sp, n3_sp])
system_sp = dpsimpy.SystemTopology(60, [n1_sp, n2_sp, n3_sp], [gen1_sp, gen2_sp, load_sp, line12_sp, line13_sp, line23_sp])

# Initialization of dynamic topology
system_sp.init_with_powerflow(system_pf)

# Logging
logger_sp = dpsimpy.Logger(sim_name_sp)
logger_sp.log_attribute("v1", "v", n1_sp)
logger_sp.log_attribute("v2", "v", n2_sp)
logger_sp.log_attribute("v3", "v", n3_sp)

logger_sp.log_attribute("v_line12", "v_intf", line12_sp)
logger_sp.log_attribute("v_line13", "v_intf", line13_sp)
logger_sp.log_attribute("v_line23", "v_intf", line23_sp)
logger_sp.log_attribute("i_line12", "i_intf", line12_sp)
logger_sp.log_attribute("i_line13", "i_intf", line13_sp)
logger_sp.log_attribute("i_line23", "i_intf", line23_sp)

logger_sp.log_attribute("Ep_gen1", "Ep_mag", gen1_sp)
logger_sp.log_attribute("v_gen1", "v_intf", gen1_sp)
logger_sp.log_attribute("i_gen1", "i_intf", gen1_sp)
logger_sp.log_attribute("wr_gen1", "w_r", gen1_sp)
logger_sp.log_attribute("delta_gen1", "delta_r", gen1_sp)

logger_sp.log_attribute("Ep_gen2", "Ep_mag", gen2_sp)
logger_sp.log_attribute("v_gen2", "v_intf", gen2_sp)
logger_sp.log_attribute("i_gen2", "i_intf", gen2_sp)
logger_sp.log_attribute("wr_gen2", "w_r", gen2_sp)
logger_sp.log_attribute("delta_gen2", "delta_r", gen2_sp)
logger_sp.log_attribute("wref_gen2", "w_ref", gen2_sp)
logger_sp.log_attribute("P_elec2", "P_elec", gen2_sp)


# Simulation
sim_sp = dpsimpy.Simulation(sim_name_sp, dpsimpy.LogLevel.debug)
sim_sp.set_system(system_sp)
sim_sp.set_time_step(time_step)
sim_sp.set_final_time(final_time)
sim_sp.set_domain(dpsimpy.Domain.SP)
sim_sp.add_logger(logger_sp)

sim_sp.run()

## SP Results

In [ ]:
work_dir = 'logs/SP_SynGenTrStab_3Bus_SteadyState_SP/'
log_name = 'SP_SynGenTrStab_3Bus_SteadyState_SP'
print(work_dir + log_name + '.csv')
ts_sp1ph_TrStab_dl= rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

## DP Results

In [ ]:
work_dir = 'logs/DP_SynGenTrStab_3Bus_SteadyState_DP/'
log_name = 'DP_SynGenTrStab_3Bus_SteadyState_DP'
print(work_dir + log_name + '.csv')
ts_dp1ph_TrStab_dl = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

## Generator 1&2 terminal voltage

In [ ]:
timestep=50e-6;
t_begin=0
t_end=30

begin_idx = int(t_begin/timestep)
end_idx= int(t_end/timestep)

f= plt.figure(figsize=(16,8))

f.add_subplot(1, 2, 1)
plt.ylabel('Generator 1 terminal voltage (V)')

for name in ['v_gen1']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' DP backshift', linestyle='--')
plt.legend()

f.add_subplot(1, 2, 2)
plt.ylabel('Generator 2 terminal voltage (V)')

for name in ['v_gen2']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' DP backshift', linestyle='--')
plt.legend()

f.show()

## Generator 1&2 terminal Current

In [ ]:
f= plt.figure(figsize=(16,8))

f.add_subplot(1, 2, 1)
plt.ylabel('Generator 1 terminal current (A)')

for name in ['i_gen1']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' DP backshift', linestyle='--')
plt.legend()

f.add_subplot(1, 2, 2)
plt.ylabel('Generator 2 terminal current (A)')

for name in ['i_gen2']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' DP backshift', linestyle='--')
plt.legend()

f.show()

## Generator 1&2 Rotor frequency

In [ ]:
f= plt.figure(figsize=(16,8))

f.add_subplot(1, 2, 1)
plt.xlabel('time (s)', fontsize=20)
plt.ylabel('Generator 1 Rotor frequency (Hz)', fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.ylim(55,65)

for name in ['wr_gen1']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*60/377, label='SP')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*60/377, label='DP', linestyle='--')
plt.legend()

f.add_subplot(1, 2, 2)
plt.xlabel('time (s)', fontsize=20)
plt.ylabel('Generator 2 Rotor frequency (Hz)', fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.ylim(55,65)

for name in ['wr_gen2']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*60/377, label='SP')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*60/377, label='DP', linestyle='--')
plt.legend()

f.show()

## Generator 1&2 Rotor angular velocity $\omega _r$

In [ ]:
f= plt.figure(figsize=(16,8))

f.add_subplot(1, 2, 1)
plt.xlabel('time (s)', fontsize=20)
plt.ylabel('Generator 1 Rotor angular velocity (1/s)', fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.ylim(360,400)

for name in ['wr_gen1']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' DP backshift', linestyle='--')
plt.legend()

f.add_subplot(1, 2, 2)
plt.xlabel('time (s)', fontsize=20)
plt.ylabel('Generator 2 Rotor angular velocity (1/s)', fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.ylim(360,400)

for name in ['wr_gen2']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' DP backshift', linestyle='--')
plt.legend()

f.show()

## Generator 1&2 Rotor angle $\delta _r$

In [ ]:
f= plt.figure(figsize=(16,8))

f.add_subplot(1, 2, 1)
plt.xlabel('time (s)', fontsize=20)
plt.ylabel('Generator 1 Rotor angle (degree)', fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.ylim(-20,20)

for name in ['delta_gen1']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*180/3.14, label=name + ' SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*180/3.14, label=name + ' DP backshift', linestyle='--')
plt.legend()

f.add_subplot(1, 2, 2)
plt.xlabel('time (s)', fontsize=20)
plt.ylabel('Generator 2 Rotor angle (degree)', fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.ylim(-20,20)

for name in ['delta_gen2']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*180/3.14, label=name + ' SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*180/3.14, label=name + ' DP backshift', linestyle='--')
plt.legend()

f.show()